In [13]:
import pandas as pd
from datetime import datetime, timedelta
import random

In [14]:
# Load the dataset (assuming the dataset is named 'data.csv')
data = pd.read_csv('examdatabase.csv')
data.head(10)

,Application_ID,D_O_B,SEX,H_Qual,TH_CENT_CH,SEC_TH_CEN,CATEGORY,Rollno,cent_allot,cent_add,examDate,batch,rep_time
0,9598673,9/13/1989,F,X,WGL,KARN,OBC,NaN,NaN,NaN,NaN,NaN,NaN
1,9657127,6/14/1989,M,GRAD,WGL,KARN,ST,NaN,NaN,NaN,NaN,NaN,NaN
2,9657164,10/13/1989,F,X,WGL,KARN,GEN,NaN,NaN,NaN,NaN,NaN,NaN
3,9657217,8/22/1985,M,GRAD,WGL,KARN,OBC,NaN,NaN,NaN,NaN,NaN,NaN
4,9657229,9/20/1991,M,X,WGL,KARN,GEN,NaN,NaN,NaN,NaN,NaN,NaN
5,9657267,11/25/1995,M,GRAD,WGL,KARN,ST,NaN,NaN,NaN,NaN,NaN,NaN
6,9657269,02-05-1997,M,X,WGL,KARN,GEN,NaN,NaN,NaN,NaN,NaN,NaN
7,9657306,5/20/1992,M,GRAD,WGL,KARN,GEN,NaN,NaN,NaN,NaN,NaN,NaN
8,9657354,02-12-2000,M,X,WGL,KARN,GEN,NaN,NaN,NaN,NaN,NaN,NaN
9,9657362,10/14/1997,M,XII,WGL,KARN,ST,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Calculate candidate count in each district based on first choice of examination city
district_candidate_count = data['TH_CENT_CH'].value_counts()
district_candidate_count

WGL     299
MAHB    289
KUN     249
GUN     198
KARN    196
KRS     171
CTT     169
VIZ     150
PRA     145
NALG    130
MED     128
ADI     123
KPM     119
ANA     107
TRI     107
KHAM     85
NEL      85
SOA      84
VIZI     84
EGOD     84
SIR      80
NIZA     73
PUD      70
KRK      69
WGOD     56
VIJ       1
PNI       1
Name: TH_CENT_CH, dtype: int64

In [4]:
# Adjust candidates to other districts based on second choices
def adjust_candidates(row):
    if row['TH_CENT_CH'] in district_candidate_count:
        return row['TH_CENT_CH']
    elif row['SEC_TH_CEN'] in district_candidate_count:
        return row['SEC_TH_CEN']
    else:
        return None
    

In [5]:
data['Final_Cent'] = data.apply(adjust_candidates, axis=1)
data.head()

,Application_ID,D_O_B,SEX,H_Qual,TH_CENT_CH,SEC_TH_CEN,CATEGORY,Rollno,cent_allot,cent_add,examDate,batch,rep_time,Final_Cent
0,9598673,9/13/1989,F,X,WGL,KARN,OBC,NaN,NaN,NaN,NaN,NaN,NaN,WGL
1,9657127,6/14/1989,M,GRAD,WGL,KARN,ST,NaN,NaN,NaN,NaN,NaN,NaN,WGL
2,9657164,10/13/1989,F,X,WGL,KARN,GEN,NaN,NaN,NaN,NaN,NaN,NaN,WGL
3,9657217,8/22/1985,M,GRAD,WGL,KARN,OBC,NaN,NaN,NaN,NaN,NaN,NaN,WGL
4,9657229,9/20/1991,M,X,WGL,KARN,GEN,NaN,NaN,NaN,NaN,NaN,NaN,WGL


In [7]:
# Create a mapping of examination city codes to district names
city_district_mapping = {
    'WGL': 'WGL',
    'MAHB': 'MAHB',
    'KUN': 'KUN',
    'GUN' : 'GUN',
    'KARN' : 'KARN',
    'KRS' : 'KRS',
    'CTT' : 'CTT',
    'VIZ' : 'VIZ',
    'PRA' : 'PRA',
    'NALG' : 'NALG',
    'MED' : 'MED',
    'ADI' : 'ADI',
    'KPM' : 'KPM',
    'ANA' : 'ANA',
    'TRI': 'TRI',
    'KHAM' : 'KHAM',
    'NEL' : 'NEL',
    'SOA' : 'SOA',
    'VIZI' : 'VIZI',
    'EGOD' : 'EGOD',
    'SIR' : 'SIR',
    'NIZA' : 'NIZA',
    'PUD' : 'PUD',
    'KRK' : 'KRK',
    'WGOD' : 'WGOD',
    'VIJ' : 'VIJ',
    'PNI' : 'PNI'
    # Add more mappings as needed
}


In [8]:
# Allocate candidates to examination centers and shifts
def allocate_centers(row):
    district = city_district_mapping[row['Final_Cent']]
    row['cent_allot'] = row['Final_Cent']
    row['cent_add'] = f'NIELIT {district}'
    return row

data = data.apply(allocate_centers, axis=1)

In [9]:
# Allocate examDate, batch, and rep_time
start_date = datetime(2023, 9, 1)
end_date = datetime(2023, 9, 30)
date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

def allocate_exam_info(row):
    row['Rollno'] = f'NL{2000000 + row.name + 1}'
    row['examDate'] = random.choice(date_range).strftime('%d-%m-%Y')
    row['batch'] = random.choice([1, 2])
    row['rep_time'] = '9:00 AM' if row['batch'] == 1 else '2:00 PM'
    return row

data = data.apply(allocate_exam_info, axis=1)

In [10]:
# Save the final examination database to a CSV file
data[['Rollno', 'cent_allot', 'cent_add', 'examDate', 'batch', 'rep_time']].to_csv('exam_database.csv', index=False)

In [11]:
# Load the CSV file into a DataFrame
data = pd.read_csv('exam_database.csv')

# Check for duplicate rows
duplicates = data[data.duplicated()]

# Display duplicate rows
print("Duplicate Rows:")
print(duplicates)


Duplicate Rows:
Empty DataFrame
Columns: [Rollno, cent_allot, cent_add, examDate, batch, rep_time]
Index: []
